# Hough Circle Transform

### Một số thư viện được sử dụng
1. OpenCV 
2. Numpy - Xử lý ma trận
3. Math - Các hàm toán học: sin, cos, deg2rad
4. Matplotlib.pyplot - biểu diễn ảnh nhanh
5. Collections.defaultdict - gọi 1 dict đặc trưng. Cụ thể là các item được gán 1 giá trị cụ thể khi được gọi ngay cả khi chưa được thêm vào trước đó. 

In [2]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
from collections import defaultdict

### Các giá trị được gọi tới cùng hàm gồm:
1. image: Ma trận biểu diễn của ảnh đầu vào
2. edge_image: ảnh đã được hiệu chỉnh bằng thư viện cv2.Candy để trích xuất đặc trưng các cạnh nổi bật
3. r_min, r_max: Cận trên và cận dưới đối với bán kính của các hình tròn có thể detect
4. delta_r, num_thetas: số lượng r và theta được tính
5. bin_threshold: ngưỡng chọn với mỗi tọa độ trên tọa độ cực 
6. post_process: biến đánh dấu việc có thực hiện so sánh giữa các hình tròn gần như nhằm lọc bớt sự nhiễu

In [3]:
def find_hough_circles(image, edge_image, r_min, r_max, delta_r, num_thetas, bin_threshold, post_process = True):
    img_height, img_width = edge_image.shape[:2]
    
    dtheta = int(360 / num_thetas)
    
    thetas = np.arange(0, 360, step = dtheta)
    
    # tập các độ dài bán kinh được xét
    rs = np.arange(r_min, r_max, step = delta_r)
    
    cos_thetas = np.cos(np.deg2rad(thetas))
    sin_thetas = np.sin(np.deg2rad(thetas))
    
    # x = x_center + r * cos(t) và y = y_center + r * sin(t),  
    # tọa độ (x_center,y_center) là tâm đường tròn bán kính r. t trong khoảng [0,2PI)
    # tính toán các giá trị r * cos(t) và r * sin(t) trước 
    circle_candidates = []
    for r in rs:
        for t in range(num_thetas):
            circle_candidates.append((r, int(r * cos_thetas[t]), int(r * sin_thetas[t])))
    
    accumulator = defaultdict(int)
    
    for y in range(img_height):
        for x in range(img_width):
            if edge_image[y][x] != 0: #không có ảnh 
                # Tính toán các hình tròn có thể đi qua điểm và biểu diễn trong không gian tọa độ cực với r và t(theta)
                for r, rcos_t, rsin_t in circle_candidates:
                    x_center = x - rcos_t
                    y_center = y - rsin_t
                    accumulator[(x_center, y_center, r)] += 1 #thêm 1 hình tròn đi qua điểm ở tọa độ cực 
    
    output_img = image.copy()
    
    out_circles = []
    
    # Sắp xếp các hình tròn đã được xác định theo chiều giảm dần số điểm đi qua
    for candidate_circle, votes in sorted(accumulator.items(), key=lambda i: -i[1]):
        x, y, r = candidate_circle
        current_vote_percentage = votes / num_thetas
        if current_vote_percentage >= bin_threshold: 
            # giảm thiểu số đường tròn không đạt ngưỡng(khi mà số điểm đi qua ít hơn với ngưỡng đã đặt)
            out_circles.append((x, y, r, current_vote_percentage))
            
    # lọc các hình tròn gần nhau để tránh sự trùng lặp với cùng 1 đối tượng
    if post_process :
        pixel_threshold = 5
        postprocess_circles = []
        for x, y, r, v in out_circles:
            # Loại các vòng quá gần nhau
            # all((x - xc) ** 2 + (y - yc) ** 2 > rc ** 2 for xc, yc, rc, v in postprocess_circles)
            # loại bỏ các cặp mà hình tròn sau gần với hình tròn trước(khoảng cách 2 hình < 5) 
            if all(abs(x - xc) > pixel_threshold or abs(y - yc) > pixel_threshold or abs(r - rc) > pixel_threshold for xc, yc, rc, v in postprocess_circles):
                postprocess_circles.append((x, y, r, v))
        out_circles = postprocess_circles
        
    # Vẽ các hình tròn định dạng vào hình mới
    for x, y, r, v in out_circles:
        output_img = cv2.circle(output_img, (x,y), r, (0,255,0), 2)
        
    return output_img, out_circles

In [4]:
# Thêm đường dẫn đến ảnh bạn muốn xử lý và cài đặt các chỉ số bạn muốn 
img_path = "D:\Python\Detect_circle\\test1.png" 
r_min = 10
r_max = 200
delta_r = 1
num_thetas = 180
bin_threshold = 0.4
min_edge_threshold = 100
max_edge_threshold = 200
input_img = cv2.imread(img_path)
    

edge_image = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
cv2.imwrite("gray1.png", edge_image)
#Lọc lấy các cạnh đặc trưng trong hình
edge_image = cv2.Canny(edge_image, min_edge_threshold, max_edge_threshold)
cv2.imwrite("edge_image1.png", edge_image)
    
if edge_image is not None:
        
    circle_img, circles = find_hough_circles(input_img, edge_image, r_min, r_max, delta_r, num_thetas, bin_threshold)
        
    if circle_img is not None:
        cv2.imwrite("circles_img1.png", circle_img)
            

### Một số thắc mắc:
1. Có hướng giải quyết triệt để cho bài toán khử nhiễu cho các ảnh đặt chồng các vật không?
2. Việc chuyển ảnh về hệ gray phát sinh 1 số vấn đề dẫn đến không tạo được sự phân biệt tại 1 số điểm? Liệu có cách nào cải tiến cho vấn đề này không?
3. Các ảnh quá lớn thường dẫn đến thời gian chạy của chương trình khá lâu. Việc sử dụng ma trận pooling để giảm kích thước dữ liệu trong mạng ConvNet có thể giảm kích thước của ảnh nhưng có làm mất đi sự phân biệt giữa vật thể với nền của ảnh không?
4. Việc chọn r_min và r_max ảnh hưởng khá lớn đến tốc độ chạy của chương trình và gây ra khá nhiều vấn đề xoay quanh việc ảnh bị nhiễu. Ngay cả với thuật toán được cài trong OpenCV thì vẫn phụ thuộc vào việc người dùng chọn thông số mindist, vậy có cách nào giải quyết triệt để vấn đề này không?